In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import warnings
warnings.filterwarnings("ignore",module='bs4')
warnings.filterwarnings("ignore",module='requests')

In [85]:
#Get the url list for scrapping
url_list = ['http://www.waste-cwin.org/node/'+str(i) for i in range(2000,3000)]
#我观察的那些焚烧厂的链接格式最后变得就是最后的这个4位数，http://www.waste-cwin.org/node/2222，比如这个2222，但是他这个4位数有的指向的不是
#焚烧厂信息，只有特定的几个是，这个很麻烦
#第二 每个焚烧厂信息完善度不一样 有的信息有的缺乏， 所以 你最好设定想要哪几个信息

In [86]:
#Ger the content given a url
def get_content(url):
    response = requests.get(url)
    if response.status_code != 200:
        return None
    else:
        page = BeautifulSoup(response.content,'html.parser')
        content = list()
        for element in page.find_all('div', class_="field-item even"):
            content.append(element.get_text().strip())
        return content

In [97]:
#Get the info for the trash processiong map in a data table

In [94]:
def get_dataframe(url_list):
    labels = ['厂名/项目名','地址','运营状况','规模','使用技术','烟气净化技术','运营时间','环评简本情况',\
              '是否属于2017年重点排污单位','监督性监测信息公布渠道', '自行监测信息公布渠道','监管记录','飞灰去向']
    info = dict([(key, []) for key in labels])
    content_list = list()
    for url in url_list:
        content_list.append(get_content(url))
    res = list()
    for val in content_list: 
        if val != None:
            res.append(val) 
    for i in range(len(res)):
        for j in range(len(res[i])):
            info[labels[j]].append(res[i][j])
    return pd.DataFrame.from_dict(info)

In [99]:
url_list = ['http://www.waste-cwin.org/node/2451','http://www.waste-cwin.org/node/2624']
data_table = get_dataframe(url_list)

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [100]:
data_table

,厂名/项目名,地址,运营状况,规模,使用技术,烟气净化技术,运营时间,环评简本情况,是否属于2017年重点排污单位,监督性监测信息公布渠道,自行监测信息公布渠道,监管记录,飞灰去向
0,日照第一生活垃圾焚烧发电厂,日照市经济技术开发区常州路9号,运行,一期600t/d 二期400t/d,机械炉排炉,SNCR脱硝+半干法脱酸+干法脱酸+活性炭吸附+布袋除尘器,一期2015年10月运行,http://huanbao.bjx.com.cn/news/20170516/825643...,是,未公开,日照市重点监控企业自动监测信息系统http://219.146.185.5:8401/web...,暂无,日照市黄山垃圾填埋场
1,伊春城市生活垃圾焚烧发电厂,"伊春循环经济园区内(伊春火车站东北侧旭日办事处铁东街),",运行,400t/d,循环流化床,半干式洗涤加布袋除尘,2014年11月运行,未公开,是,伊春市环保局官网http://www.yhb.gov.cn/docweb/docList.a...,黑龙江省重点监控企业环境自行监测信息发布平台http://1.189.191.146:800...,http://www.ipe.org.cn/IndustryRecord/regulator...,伊春市生活垃圾填埋场
